In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

use_own_data = False

if use_own_data:
    path = '/home/florian/Desktop/Data/'
    example_img = plt.imread(path+'IMG/'+os.listdir(path+'IMG')[0])
else:
    path = '../data/data/'
    example_img = plt.imread(path+'IMG/center_2016_12_01_13_30_48_287.jpg')

plt.imshow(example_img)

img_shape = example_img.shape
print('img_shape =',img_shape)

top_crop = 60
bottom_crop = 0
left_crop = 0
right_crop = 0

x_val = np.arange(0,example_img.shape[1])
plt.plot(x_val, np.ones(len(x_val))*top_crop, 'r')

## Load images

In [ ]:
import csv

samples = []
_angles = []
with open(path+'driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # skip headers in first row
    for line in reader:
        samples.append(line)
        _angles.append(float(line[3]))
        
print('number of images (initial data set): ',len(samples))

In [ ]:
# # display example images and angles:
# for i in range(3):
#     rand_num = np.random.randint(len(samples))
#     rand_sample = samples[rand_num]
#     center_img_name = path+'IMG/'+rand_sample[0].split('/')[-1]
#     center_img = plt.imread(center_img_name)
#     center_angle = float(rand_sample[3])
#     print(center_angle)
#     plt.title('Center angle: ', center_angle)
#     plt.imshow(center_img); plt.show()

## Exploration: Display angle distribution and angle over time

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,6))
# sns.distplot(angles, kde=False)
plt.hist(_angles, 30)
plt.title('Initial data: steering angle distribution (absolute)')
plt.xlabel('Steering angle')
plt.show()

# %matplotlib notebook
fig = plt.figure(figsize=(10,6))
plt.plot(_angles)
plt.title('Angle over time')
plt.xlabel('Sample')
plt.ylabel('Angle')
plt.show()

## Augmentation

In [ ]:
with open(path+'driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # skip headers in first row
    for line in reader:
#         for i in range(2):
        samples.append(line)
        _angles.append(float(line[3]))
##         if abs(float(line[3]))>.1:
#         _angle_val = abs(float(line[3]))
#         if _angle_val==0:
#             samples.append(line)
#             _angles.append(float(line[3]))
#         if _angle_val!=0:
#             # Add identical images multiple times for all angles != 0. The model includes a noise layer, which adds small changes to all images.
# #             for i in range(int(np.round(_angle_val*10))):
#             for i in range(3)
#                 samples.append(line)
#                 _angles.append(float(line[3]))
                
print('number of images (augmented data set): ',len(samples))

In [ ]:
# %matplotlib inline
# fig = plt.figure(figsize=(10,6))
# # sns.distplot(angles, kde=False)
# plt.hist(_angles, 30)
# plt.title('Augmented data: steering angle distribution (absolute)')
# plt.xlabel('Steering angle')
# plt.show()

# # %matplotlib notebook
# fig = plt.figure(figsize=(10,6))
# plt.plot(_angles)
# plt.title('Angle over time')
# plt.xlabel('Sample')
# plt.ylabel('Angle')
# plt.show()

## Train-test-split

In [ ]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

print('number of training samples: ', len(train_samples))
print('number of validation samples: ', len(validation_samples))

In [ ]:
import cv2
import numpy as np
import sklearn
from scipy import ndimage # for importing image as RGB

angle_offset = .25

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1:
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                
                center_img_name = path+'IMG/'+batch_sample[0].split('/')[-1]
                center_img = plt.imread(center_img_name)
                center_angle = float(batch_sample[3])
                images.append(center_img)
                angles.append(center_angle)
                images.append(np.fliplr(center_img))
                angles.append(-center_angle)
                
                left_img_name = path+'IMG/'+batch_sample[1].split('/')[-1]
                left_img = plt.imread(left_img_name)
                left_angle = center_angle + angle_offset
                images.append(left_img)
                angles.append(left_angle)
                images.append(np.fliplr(left_img))
                angles.append(-left_angle)
                
                right_img_name = path+'IMG/'+batch_sample[2].split('/')[-1]
                right_img = plt.imread(right_img_name)
                right_angle = center_angle - angle_offset
                images.append(right_img)
                angles.append(right_angle)
                images.append(np.fliplr(right_img))
                angles.append(-right_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

            

In [ ]:
# # augmentation
# import numpy as np
# image_flipped = np.fliplr(image)
# measurement_flipped = -measurement

In [ ]:
# # use all three images
#    with open(csv_file, 'r') as f:
#         reader = csv.reader(f)
#         for row in reader:
#             steering_center = float(row[3])

#             # create adjusted steering measurements for the side camera images
#             correction = 0.2 # this is a parameter to tune
#             steering_left = steering_center + correction
#             steering_right = steering_center - correction

#             # read in images from center, left and right cameras
#             path = "..." # fill in the path to your training IMG directory
#             img_center = process_image(np.asarray(Image.open(path + row[0])))
#             img_left = process_image(np.asarray(Image.open(path + row[1])))
#             img_right = process_image(np.asarray(Image.open(path + row[2])))

#             # add images and angles to data set
#             car_images.extend(img_center, img_left, img_right)
#             steering_angles.extend(steering_center, steering_left, steering_right)


In [ ]:
import tensorflow as tf
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())
print(tf.test.is_built_with_cuda())

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Lambda, Dense, Activation, Flatten, Dropout
from keras.layers import Cropping2D, GaussianNoise
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers
    
# Set batch size
batch_size = 32
l2_penal = .0 # .0001: overfitting after 8 epochs
dr1 = 0.3
dr2 = 0.2
dr3 = 0.1

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)


row, col, ch = img_shape[0], img_shape[1], img_shape[2]  # Trimmed image format

model = Sequential()
# normalize data 
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(row, col, ch), output_shape=(row, col, ch)))
# crop images
model.add(Cropping2D(cropping=((top_crop, bottom_crop),(left_crop, right_crop))))
# add noise layer
model.add(GaussianNoise(.1))
# add convolutional layers
model.add(Conv2D(24, kernel_size=(5, 5), strides=(2,2), activation='relu'))#, subsample=(2, 2)))
model.add(Conv2D(36, kernel_size=(5, 5), strides=(2,2), activation='relu'))#, subsample=(2, 2)))
model.add(Conv2D(48, kernel_size=(5, 5), strides=(2,2), activation='relu'))#, subsample=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), strides=(1,1), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), strides=(1,1), activation='relu'))
# add flatten layer
model.add(Flatten())
# add fully connected layers plus dropout
model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(l2_penal)))
model.add(Dropout(rate=dr1))
model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(l2_penal)))
model.add(Dropout(rate=dr2))
model.add(Dense(10, activation='relu', kernel_regularizer=regularizers.l2(l2_penal)))
model.add(Dropout(rate=dr3))
model.add(Dense(1))

model.summary()

In [ ]:
save_path = './model.h5'
num_epochs = 30

# compile method configures learning process
model.compile(optimizer='adam', loss='mse')
# model.compile(optimizer=Adam(learning_rate), loss='mse')# metrics=['accuracy']

checkpoint = ModelCheckpoint(filepath=save_path, monitor='val_loss', save_best_only=True)
stopper = EarlyStopping(monitor='val_loss', min_delta=0.0003, patience=3)

history = model.fit_generator(train_generator, 
                    steps_per_epoch=np.ceil(len(train_samples)/batch_size), 
                    validation_data=validation_generator, 
                    validation_steps=np.ceil(len(validation_samples)/batch_size), 
                    callbacks=[checkpoint, stopper],
                    epochs=num_epochs,
                    verbose=1)

# model.save('model.h5')

In [ ]:
print(history.history.keys())
### plot the training and validation loss for each epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()